In [1]:
import nltk
from nltk.corpus import twitter_samples

In [2]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Sandhita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sandhita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sandhita\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Sandhita\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sandhita\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Sandhita\AppData\Roaming\nltk_data...
[nltk_data]   Package 

True

In [3]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')


In [4]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')


In [5]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import re, string
from nltk.corpus import stopwords
from nltk import FreqDist, classify, NaiveBayesClassifier
from nltk.tokenize import word_tokenize
import random

In [6]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

In [7]:
def remove_noise(tweet_tokens, stop_words = ()): 
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub(r'http[s]?://(?:[a-zA-Z0-9$-_@.&+#]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', token)
        token = re.sub("(@[A-Za-z0-9_]+)", "", token)
        
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [8]:
stop_words = stopwords.words('english')

In [9]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

In [10]:
positive_cleaned_tokens_list = [remove_noise(tokens, stop_words) for tokens in positive_tweet_tokens]
negative_cleaned_tokens_list = [remove_noise(tokens, stop_words) for tokens in negative_tweet_tokens]


In [11]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [12]:
all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [13]:
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))


[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 332), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [14]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [15]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)


In [16]:
positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

In [17]:
dataset = positive_dataset + negative_dataset
random.shuffle(dataset)

In [18]:
train_data = dataset[:7000]
test_data = dataset[7000:]

In [19]:
classifier = NaiveBayesClassifier.train(train_data)
print("Accuracy is:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Accuracy is: 0.9953333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2089.5 : 1.0
                      :) = True           Positi : Negati =    990.6 : 1.0
                     sad = True           Negati : Positi =     33.1 : 1.0
                follower = True           Positi : Negati =     20.3 : 1.0
                     bam = True           Positi : Negati =     19.4 : 1.0
               community = True           Positi : Negati =     16.7 : 1.0
              appreciate = True           Positi : Negati =     14.1 : 1.0
                    glad = True           Positi : Negati =     13.6 : 1.0
                    miss = True           Negati : Positi =     13.3 : 1.0
                    kill = True           Negati : Positi =     12.5 : 1.0
None


In [20]:
custom_tweet = input("Enter a tweet: ")
print("Tweet: ",custom_tweet)
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(classifier.classify(dict([token, True] for token in custom_tokens)))

Tweet:  Yay! Today is a holiday
Positive
